# 4.10 Part 1: Instacart

### This script contains the following points:
#### 01. Import libraries and data (incl. security implications/PII)
#### 02. Consistency Checks on df_ords_prods_customers and df_dep
####       a. Merge department_wrangled.csv to df and save as df_all
#### 03. Create & Explore Regions
#### 04. Create Exclusion Flag for Low-Activity customers
#### 05. Create Customer Profiles/Flags
#### 06. Export data df_all, df_excluded, and df_low activity as pkl files

# 01 Import libraries & data

In [1]:
#import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import scipy

In [2]:
#create data path
path = r'C:\Users\fa_an\OneDrive\CareerFoundry\Tasks\Data Analytics Immersion\Tasks 4.1-4.10\02_2024 Instacart Basket Analysis'

In [3]:
#import data - df_all (order_products_customers w/ revisions)
df_ords_prods_customers = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_customers_rev2.pkl'))

#### Security Implications already considered based on customer/PII data. First Name and Surnam not saved/not merged with ords_prods_customers_rev.pkl

In [4]:
#import data - departments_wrangled.csv to get dept names
df_dep = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'departments_wrangled.csv'), index_col = 0)

In [5]:
pd.set_option('display.max_columns', None)

# 02 Consistency Checks on df_all and df_dep

In [6]:
df_ords_prods_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32404859 entries, 0 to 32404858
Data columns (total 27 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   order_id                    int32  
 1   user_id                     int32  
 2   order_number                int8   
 3   orders_day_of_week          int8   
 4   order_hour_of_day           int8   
 5   days_since_prior_order      float16
 6   product_id                  int32  
 7   add_to_cart_order           uint8  
 8   reordered                   int8   
 9   product_name                object 
 10  department_id               int8   
 11  prices                      float32
 12  price_label                 object 
 13  busiest_days                object 
 14  max_order                   int8   
 15  loyalty_flag                object 
 16  mean_product_price          float32
 17  spending_flag               object 
 18  median_days_between_orders  float16
 19  order_frequency_fla

In [7]:
df_ords_prods_customers.head(20)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,department_id,prices,price_label,busiest_days,max_order,loyalty_flag,mean_product_price,spending_flag,median_days_between_orders,order_frequency_flag,Gender,State,Age,total_dependants,family_status,income,sum_order
0,2539329,1,1,2,8,NaN,196,1,0,Soda,7,9.0,Mid-range product,Regularly busy,10,New Customer,6.367796,Low Spender,20.5,Non-frequent customer,Female,Alabama,31,3,married,40423,31.600000
1,2398795,1,2,3,7,15.0,196,1,1,Soda,7,9.0,Mid-range product,Slowest days,10,New Customer,6.367796,Low Spender,20.5,Non-frequent customer,Female,Alabama,31,3,married,40423,35.400002
2,473747,1,3,3,12,21.0,196,1,1,Soda,7,9.0,Mid-range product,Slowest days,10,New Customer,6.367796,Low Spender,20.5,Non-frequent customer,Female,Alabama,31,3,married,40423,28.200001
3,2254736,1,4,4,7,29.0,196,1,1,Soda,7,9.0,Mid-range product,Slowest days,10,New Customer,6.367796,Low Spender,20.5,Non-frequent customer,Female,Alabama,31,3,married,40423,26.000000
4,431534,1,5,4,15,28.0,196,1,1,Soda,7,9.0,Mid-range product,Slowest days,10,New Customer,6.367796,Low Spender,20.5,Non-frequent customer,Female,Alabama,31,3,married,40423,57.000000
5,3367565,1,6,2,7,19.0,196,1,1,Soda,7,9.0,Mid-range product,Regularly busy,10,New Customer,6.367796,Low Spender,20.5,Non-frequent customer,Female,Alabama,31,3,married,40423,25.000000
6,550135,1,7,1,9,20.0,196,1,1,Soda,7,9.0,Mid-range product,Busiest days,10,New Customer,6.367796,Low Spender,20.5,Non-frequent customer,Female,Alabama,31,3,married,40423,29.000000
7,3108588,1,8,1,14,14.0,196,2,1,Soda,7,9.0,Mid-range product,Busiest days,10,New Customer,6.367796,Low Spender,20.5,Non-frequent customer,Female,Alabama,31,3,married,40423,40.200001
8,2295261,1,9,1,16,0.0,196,4,1,Soda,7,9.0,Mid-range product,Busiest days,10,New Customer,6.367796,Low Spender,20.5,Non-frequent customer,Female,Alabama,31,3,married,40423,40.200001
9,2550362,1,10,4,8,30.0,196,1,1,Soda,7,9.0,Mid-range product,Slowest days,10,New Customer,6.367796,Low Spender,20.5,Non-frequent customer,Female,Alabama,31,3,married,40423,63.099998


In [8]:
df_dep.head(21)

,department
1,frozen
2,other
3,bakery
4,produce
5,alcohol
6,international
7,beverages
8,pets
9,dry goods pasta
10,bulk


In [9]:
df_dep.shape

(21, 1)

# 02a Merge department_wrangled.csv to df_ords_prods_customers

In [10]:
#merge and name as df_all
df_all = df_ords_prods_customers.merge(df_dep, left_on='department_id', right_index=True)

In [11]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32404859 entries, 0 to 32403040
Data columns (total 28 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   order_id                    int32  
 1   user_id                     int32  
 2   order_number                int8   
 3   orders_day_of_week          int8   
 4   order_hour_of_day           int8   
 5   days_since_prior_order      float16
 6   product_id                  int32  
 7   add_to_cart_order           uint8  
 8   reordered                   int8   
 9   product_name                object 
 10  department_id               int8   
 11  prices                      float32
 12  price_label                 object 
 13  busiest_days                object 
 14  max_order                   int8   
 15  loyalty_flag                object 
 16  mean_product_price          float32
 17  spending_flag               object 
 18  median_days_between_orders  float16
 19  order_frequency_flag    

In [12]:
df_all.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,department_id,prices,price_label,busiest_days,max_order,loyalty_flag,mean_product_price,spending_flag,median_days_between_orders,order_frequency_flag,Gender,State,Age,total_dependants,family_status,income,sum_order,department
0,2539329,1,1,2,8,NaN,196,1,0,Soda,7,9.0,Mid-range product,Regularly busy,10,New Customer,6.367796,Low Spender,20.5,Non-frequent customer,Female,Alabama,31,3,married,40423,31.600000,beverages
1,2398795,1,2,3,7,15.0,196,1,1,Soda,7,9.0,Mid-range product,Slowest days,10,New Customer,6.367796,Low Spender,20.5,Non-frequent customer,Female,Alabama,31,3,married,40423,35.400002,beverages
2,473747,1,3,3,12,21.0,196,1,1,Soda,7,9.0,Mid-range product,Slowest days,10,New Customer,6.367796,Low Spender,20.5,Non-frequent customer,Female,Alabama,31,3,married,40423,28.200001,beverages
3,2254736,1,4,4,7,29.0,196,1,1,Soda,7,9.0,Mid-range product,Slowest days,10,New Customer,6.367796,Low Spender,20.5,Non-frequent customer,Female,Alabama,31,3,married,40423,26.000000,beverages
4,431534,1,5,4,15,28.0,196,1,1,Soda,7,9.0,Mid-range product,Slowest days,10,New Customer,6.367796,Low Spender,20.5,Non-frequent customer,Female,Alabama,31,3,married,40423,57.000000,beverages


In [13]:
df_all['department'].shape

(32404859,)

In [14]:
# Define the mapping of integers to day names
day_mapping = {0: 'Saturday', 1: 'Sunday', 2: 'Monday', 3: 'Tuesday', 4: 'Wednesday', 5: 'Thursday', 6: 'Friday'}

# Convert to categorical
df_all['orders_day_of_week'] = df_all['orders_day_of_week'].map(day_mapping).astype('category')

In [15]:
df_all['department'].value_counts(dropna = False)

department
produce            9479291
dairy eggs         5398747
snacks             2887550
beverages          2688123
frozen             2234743
pantry             1875369
bakery             1172428
canned goods       1068058
deli               1051249
dry goods pasta     866627
household           738666
meat seafood        708927
breakfast           703033
personal care       447572
babies              423802
international       269253
alcohol             153696
pets                 97716
missing              69145
other                36291
bulk                 34573
Name: count, dtype: int64

# 03 Create Regions & explore data
The Instacart officers are interested in comparing customer behavior in different geographic areas. Create a regional segmentation of the data. You’ll need to create a “Region” column based on the “State” column from your customers data set.

In [16]:
#create regions using categories
Northeast = ['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 'Connecticut', 'New York', 'Pennsylvania', 'New Jersey']
Midwest = ['Wisconsin', 'Michigan', 'Illinois', 'Indiana', 'Ohio', 'North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 'Minnesota', 'Iowa', 'Missouri']
South = ['Delaware', 'Maryland', 'District of Columbia', 'Virginia', 'West Virginia', 'North Carolina', 'South Carolina', 'Georgia', 'Florida', 'Kentucky', 'Tennessee', 'Mississippi', 'Alabama', 'Oklahoma', 'Texas', 'Arkansas', 'Louisiana']
West = ['Idaho', 'Montana', 'Wyoming', 'Nevada', 'Utah', 'Colorado', 'Arizona', 'New Mexico', 'Alaska', 'Washington', 'Oregon', 'California', 'Hawaii']

In [17]:
#create region column and use categories
df_all['Region'] = pd.Categorical(np.select(
        [df_all['State'].isin(Northeast),
         df_all['State'].isin(Midwest),
         df_all['State'].isin(South),
         df_all['State'].isin(West)],
        ['Northeast', 'MidWest', 'South', 'West'],
        default='Other'))

In [18]:
df_all[['State', 'Region']].head(15)

,State,Region
0,Alabama,South
1,Alabama,South
2,Alabama,South
3,Alabama,South
4,Alabama,South
5,Alabama,South
6,Alabama,South
7,Alabama,South
8,Alabama,South
9,Alabama,South


In [19]:
df_all['Region'].value_counts(dropna = False)

Region
South        10791885
West          8292913
MidWest       7597325
Northeast     5722736
Name: count, dtype: int64

#### By region, the majority of customers are in the South. The South does have more states in it's region. To verify, the region columns have all cells filled, and the length matches up with the rest of the dataframe.

In [20]:
df_all['Region'].shape

(32404859,)

#### Determine whether there’s a difference in spending habits between the different U.S. regions. (Hint: You can do this by crossing the variable you just created with the spending flag.)

In [21]:
#total users/customers within each region
df_all.groupby('Region').agg({'user_id': 'nunique'}).sort_values(by='user_id', ascending=False)

C:\Users\fa_an\AppData\Local\Temp\ipykernel_18680\234091655.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_all.groupby('Region').agg({'user_id': 'nunique'}).sort_values(by='user_id', ascending=False)


,user_id
Region,
South,68737
West,52565
MidWest,48519
Northeast,36388


#### As related to above, and to confirm the South has the most customers (verified by unique user_id). Total of 206,209 customers (which matches the original customers data frame).

In [22]:
#mean/min/max of product price by region
df_all.groupby('Region').agg({'prices': ['mean', 'min', 'max']})

C:\Users\fa_an\AppData\Local\Temp\ipykernel_18680\682799790.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_all.groupby('Region').agg({'prices': ['mean', 'min', 'max']})


prices           
               mean  min   max
Region                        
MidWest    7.792584  1.0  25.0
Northeast  7.782409  1.0  25.0
South      7.794918  1.0  25.0
West       7.790356  1.0  25.0

In [23]:
#total loyalty_flag customers by region (adds up to 206,209)
df_all.groupby(['Region', 'loyalty_flag'])['user_id'].nunique().unstack(fill_value=0)

C:\Users\fa_an\AppData\Local\Temp\ipykernel_18680\3972471917.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_all.groupby(['Region', 'loyalty_flag'])['user_id'].nunique().unstack(fill_value=0)


loyalty_flag,Loyal Customer,New Customer,Regular Customer
Region,,,
MidWest,3991,26333,18195
Northeast,3027,19869,13492
South,5637,37491,25609
West,4362,28635,19568


#### This shows that the most Loyal, Regular, and New customers are located in the South. The loyalty flag was set to show the number of orders a customer has placed, so just to prove that there's lots of orders placed from the South region.

In [24]:
#spending_flag customers by user_id in each region (adds up to 206,209)
df_all.groupby(['Region', 'spending_flag'])['user_id'].nunique().unstack(fill_value=0)

C:\Users\fa_an\AppData\Local\Temp\ipykernel_18680\2252682862.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_all.groupby(['Region', 'spending_flag'])['user_id'].nunique().unstack(fill_value=0)


spending_flag,High Spender,Low Spender
Region,,
MidWest,1263,47256
Northeast,883,35505
South,1817,66920
West,1394,51171


#### This shows that the South has the most High Spenders (avg product price is >= to 10) as well as the most Low Spenders too! The Northeast region is the lowest in High Spenders.

In [25]:
#total sales by region using the 'prices' column
df_all.groupby('Region')['prices'].sum().sort_values(ascending=False)

C:\Users\fa_an\AppData\Local\Temp\ipykernel_18680\3370463919.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_all.groupby('Region')['prices'].sum().sort_values(ascending=False)


Region
South        84107912.0
West         64595128.0
MidWest      59192976.0
Northeast    44530104.0
Name: prices, dtype: float32

#### The South region has spent the most compared to the rest of the US. The lowest sales are in the Northeast.

In [26]:
#grand total of sales to use for calculated figures
df_all['prices'].sum()

252426240.0

In [27]:
#total # of orders by region
df_all.groupby('Region')['order_id'].nunique().sort_values(ascending=False)

C:\Users\fa_an\AppData\Local\Temp\ipykernel_18680\691950656.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_all.groupby('Region')['order_id'].nunique().sort_values(ascending=False)


Region
South        1071562
West          821686
MidWest       754275
Northeast     567162
Name: order_id, dtype: int64

#### Just to confirm that the South has the most # of orders by region with the Northeast with the least.

In [28]:
#grand total of orders (count) to use for calculated figures
df_all['order_id'].nunique()

3214685

In [29]:
# Set the custom formatter for floating-point numbers
pd.options.display.float_format = '{:,.0f}'.format

In [30]:
#avg income per user_id in each Region (is this correct?) >test with crosstab
df_all.groupby(['Region', 'user_id'])['income'].mean().groupby('Region').mean()

C:\Users\fa_an\AppData\Local\Temp\ipykernel_18680\3579328031.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_all.groupby(['Region', 'user_id'])['income'].mean().groupby('Region').mean()


Region
MidWest     94,867
Northeast   94,974
South       94,439
West        94,434
Name: income, dtype: float64

In [31]:
#orders grouped by region and day_of_week to show popular days for shopping
df_all.groupby(['Region', 'orders_day_of_week'])['order_id'].nunique().unstack(fill_value=0)

C:\Users\fa_an\AppData\Local\Temp\ipykernel_18680\1798651776.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_all.groupby(['Region', 'orders_day_of_week'])['order_id'].nunique().unstack(fill_value=0)


orders_day_of_week,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
Region,,,,,,,
MidWest,98391,103910,131163,130433,100014,96657,93707
Northeast,73165,77975,98424,98515,75147,73042,70894
South,139811,147394,185044,186085,141844,137507,133877
West,107461,112650,143114,141646,108938,105165,102712


In [32]:
#orders grouped by region and hour_of_day to show popular time of day for shopping
df_all.groupby(['Region', 'order_hour_of_day'])['user_id'].nunique().unstack(fill_value=0)

C:\Users\fa_an\AppData\Local\Temp\ipykernel_18680\3122092206.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_all.groupby(['Region', 'order_hour_of_day'])['user_id'].nunique().unstack(fill_value=0)


order_hour_of_day,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
Region,,,,,,,,,,,,,,,,,,,,,,,,
MidWest,3638,2116,1320,952,1028,1562,4304,10429,17308,22719,25505,26650,26529,26951,26982,27174,26261,23540,20035,16505,13178,10290,8381,5756
Northeast,2671,1595,1000,752,753,1224,3198,7781,13063,16954,19099,19782,20075,20212,20349,20299,19766,17458,14959,12411,9876,7672,6270,4402
South,5133,2886,1862,1446,1423,2324,6098,14743,24704,32244,36135,37651,37690,37960,38200,38374,37367,33331,28284,23512,18644,14415,11795,8414
West,3943,2210,1423,1087,1085,1716,4623,11294,18842,24591,27805,28783,28847,29192,29417,29248,28346,25300,21753,17781,14302,11073,9044,6294


In [33]:
#Customer totals by region: user_id grouped by region
df_all.groupby('Region').agg({'user_id': 'nunique'}).sort_values(by='user_id', ascending=False)

C:\Users\fa_an\AppData\Local\Temp\ipykernel_18680\90421716.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_all.groupby('Region').agg({'user_id': 'nunique'}).sort_values(by='user_id', ascending=False)


,user_id
Region,
South,68737
West,52565
MidWest,48519
Northeast,36388


# 04 Create Exclusion Flag
#### for LOW ACTIVITY CUSTOMERS
The Instacart CFO isn’t interested in customers who don’t generate much revenue for the app. Create an exclusion flag for low-activity customers (customers with less than 5 orders) and exclude them from the data. Make sure you export this sample.

In [34]:
# Create exclusion flag and create column
df_all.loc[df_all['max_order'] < 5, 'exclusion_flag'] = 1

In [35]:
df_all.loc[df_all['max_order'] >= 5, 'exclusion_flag'] = 0

In [36]:
# Set display option to suppress scientific notation
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [37]:
df_all['exclusion_flag'] = df_all['exclusion_flag'].astype('int8')

In [38]:
# check to see that all records were allocated
df_all['exclusion_flag'].value_counts(dropna=False)

exclusion_flag
0    30964564
1     1440295
Name: count, dtype: int64

In [39]:
#create low_activity df to export sample per Instacart CFO
df_low_activity = df_all[df_all['exclusion_flag'] == 1]

In [40]:
df_low_activity.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1440295 entries, 1510 to 32403040
Data columns (total 30 columns):
 #   Column                      Non-Null Count    Dtype   
---  ------                      --------------    -----   
 0   order_id                    1440295 non-null  int32   
 1   user_id                     1440295 non-null  int32   
 2   order_number                1440295 non-null  int8    
 3   orders_day_of_week          1440295 non-null  category
 4   order_hour_of_day           1440295 non-null  int8    
 5   days_since_prior_order      1025482 non-null  float16 
 6   product_id                  1440295 non-null  int32   
 7   add_to_cart_order           1440295 non-null  uint8   
 8   reordered                   1440295 non-null  int8    
 9   product_name                1440295 non-null  object  
 10  department_id               1440295 non-null  int8    
 11  prices                      1440045 non-null  float32 
 12  price_label                 1440295 non-nul

In [41]:
#exclude low-activity customers in new df_excluded
df_excluded = df_all[df_all['exclusion_flag'] == 0]

In [42]:
df_excluded.shape

(30964564, 30)

In [43]:
df_excluded.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,department_id,prices,price_label,busiest_days,max_order,loyalty_flag,mean_product_price,spending_flag,median_days_between_orders,order_frequency_flag,Gender,State,Age,total_dependants,family_status,income,sum_order,department,Region,exclusion_flag
0,2539329,1,1,Monday,8,NaN,196,1,0,Soda,7,9.00,Mid-range product,Regularly busy,10,New Customer,6.37,Low Spender,20.50,Non-frequent customer,Female,Alabama,31,3,married,40423,31.60,beverages,South,0
1,2398795,1,2,Tuesday,7,15.00,196,1,1,Soda,7,9.00,Mid-range product,Slowest days,10,New Customer,6.37,Low Spender,20.50,Non-frequent customer,Female,Alabama,31,3,married,40423,35.40,beverages,South,0
2,473747,1,3,Tuesday,12,21.00,196,1,1,Soda,7,9.00,Mid-range product,Slowest days,10,New Customer,6.37,Low Spender,20.50,Non-frequent customer,Female,Alabama,31,3,married,40423,28.20,beverages,South,0
3,2254736,1,4,Wednesday,7,29.00,196,1,1,Soda,7,9.00,Mid-range product,Slowest days,10,New Customer,6.37,Low Spender,20.50,Non-frequent customer,Female,Alabama,31,3,married,40423,26.00,beverages,South,0
4,431534,1,5,Wednesday,15,28.00,196,1,1,Soda,7,9.00,Mid-range product,Slowest days,10,New Customer,6.37,Low Spender,20.50,Non-frequent customer,Female,Alabama,31,3,married,40423,57.00,beverages,South,0


In [44]:
df_excluded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30964564 entries, 0 to 32380882
Data columns (total 30 columns):
 #   Column                      Dtype   
---  ------                      -----   
 0   order_id                    int32   
 1   user_id                     int32   
 2   order_number                int8    
 3   orders_day_of_week          category
 4   order_hour_of_day           int8    
 5   days_since_prior_order      float16 
 6   product_id                  int32   
 7   add_to_cart_order           uint8   
 8   reordered                   int8    
 9   product_name                object  
 10  department_id               int8    
 11  prices                      float32 
 12  price_label                 object  
 13  busiest_days                object  
 14  max_order                   int8    
 15  loyalty_flag                object  
 16  mean_product_price          float32 
 17  spending_flag               object  
 18  median_days_between_orders  float16 
 19  ord

# 05 Create Customer Profiles/Flags
The marketing and business strategy units at Instacart want to create more-relevant marketing strategies for different products and are, thus, curious about customer profiling in their database. Create a profiling variable based on age, income, certain goods in the “department_id” column, and number of dependents. You might also use the “orders_day_of_week” and “order_hour_of_day” columns if you can think of a way they would impact customer profiles. (Hint: As an example, try thinking of what characteristics would lead you to the profile “Single adult” or “Young parent.”)

In [45]:
df_excluded['department'].value_counts(dropna=False)

department
produce            9079273
dairy eggs         5177182
snacks             2766406
beverages          2571901
frozen             2121731
pantry             1782705
bakery             1120828
canned goods       1012074
deli               1003834
dry goods pasta     822136
household           699857
meat seafood        674781
breakfast           670850
personal care       424306
babies              410392
international       255991
alcohol             144627
pets                 93060
missing              64768
other                34411
bulk                 33451
Name: count, dtype: int64

In [46]:
# Group by 'department' and 'region', then aggregate the sum of prices
sum_prices_by_department_region = df_excluded.groupby(['department', 'Region'])['prices'].sum().reset_index()

# Sort the result by 'department' and 'region' in descending order
sum_prices_by_department_region_sorted = sum_prices_by_department_region.sort_values(by=['department', 'Region'], ascending=False)

C:\Users\fa_an\AppData\Local\Temp\ipykernel_18680\2804122596.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sum_prices_by_department_region = df_excluded.groupby(['department', 'Region'])['prices'].sum().reset_index()


In [47]:
# Set the display option to show all rows
pd.set_option('display.max_rows', None)

In [48]:
#use crosstabs in excel
crosstab = pd.crosstab(index=sum_prices_by_department_region['department'], 
                        columns=sum_prices_by_department_region['Region'], 
                        values=sum_prices_by_department_region['prices'], 
                        aggfunc='sum')

# Sort the crosstab by 'department' and 'region' in descending order
crosstab_sorted = crosstab.sort_values(by=['department'], ascending=False)

In [49]:
crosstab_sorted

Region,MidWest,Northeast,South,West
department,,,,
snacks,2758447.50,2105588.00,3918338.75,3045546.00
produce,17034110.00,12708319.00,24167140.00,18546192.00
pets,166938.80,140749.70,234712.59,191608.59
personal care,804340.69,597516.38,1138407.62,853650.31
pantry,3368091.25,2502665.00,4744687.50,3672012.50
other,57296.80,42957.20,78757.40,60557.80
missing,134758.70,99003.70,187790.70,139380.20
meat seafood,2574335.75,1930068.75,3657937.50,2836611.50
international,463167.19,342749.50,655481.12,504305.09


In [50]:
#copy crosstab data and paste into excel
crosstab.to_clipboard()

In [51]:
# Create the crosstab
crosstab = pd.crosstab(index=df_excluded['income'], 
                       columns='mean_sum_order', 
                       values=df_excluded['sum_order'], 
                       aggfunc='mean')

In [52]:
crosstab.to_clipboard()

In [53]:
#Review 'Age' and Create Age_Range column, might make categories to fit common categories, not just based on quartiles.
df_excluded['Age'].describe()

count   30964564.00
mean          49.47
std           18.49
min           18.00
25%           33.00
50%           49.00
75%           65.00
max           81.00
Name: Age, dtype: float64

In [55]:
#Create Age_Range column (0=Young, 1=Adult, 2=Middle-Aged, 3=Senior)
df_excluded.loc[df_excluded['Age'] <= 21, 'Age_Range'] = 0
df_excluded.loc[(df_excluded['Age'] >= 21) & (df_excluded['Age'] <= 44), 'Age_Range'] = 1
df_excluded.loc[(df_excluded['Age'] >= 45) & (df_excluded['Age'] <= 64), 'Age_Range'] = 2
df_excluded.loc[df_excluded['Age'] >= 65, 'Age_Range'] = 3

df_excluded['Age_Range'].value_counts(dropna = False)

Age_Range
1.00    11624241
2.00     9690051
3.00     8195544
0.00     1454728
Name: count, dtype: int64

In [56]:
#Adults are the largest part of the customer base, accounting for 11,135,125, approx 36%.

In [57]:
#Review 'income' and create income range
df_excluded['income'].describe()

count   30964564.00
mean       99675.87
std        43141.87
min        25903.00
25%        67292.00
50%        96765.00
75%       128102.00
max       593901.00
Name: income, dtype: float64

In [59]:
#income range parameters based on quartiles and create new column
df_excluded.loc[df_excluded['income'] < 67292, 'Income_Range'] = 1
df_excluded.loc[(df_excluded['income'] >= 67292) & (df_excluded['income'] <= 96764), 'Income_Range'] = 2
df_excluded.loc[(df_excluded['income'] >= 96765) & (df_excluded['income'] <= 128101), 'Income_Range'] = 3
df_excluded.loc[df_excluded['income'] >= 128102, 'Income_Range'] = 4

In [60]:
df_excluded['Income_Range'].value_counts(dropna=False)

Income_Range
3.00    7742166
4.00    7741261
1.00    7741005
2.00    7740132
Name: count, dtype: int64

In [62]:
#Set'pet_items' flag
df_excluded.loc[:, 'pet_items'] = [1 if x == 'pets' else 0 for x in df_excluded['department']]

In [64]:
#apply it based on user_id
df_excluded.loc[:, 'pet_items'] = df_excluded.groupby(['user_id'])['pet_items'].transform('max')

In [66]:
#Set 'baby_items' flag
df_excluded.loc[:, 'baby_items'] = [1 if x == 'babies' else 0 for x in df_excluded['department']]

In [67]:
#apply it based on user_id
df_excluded.loc[:, 'baby_items'] = df_excluded.groupby(['user_id'])['baby_items'].transform('max')

In [69]:
#Set 'snacks' flag
df_excluded.loc[:, 'snacks'] = [1 if x == 'snacks' else 0 for x in df_excluded['department']]

In [70]:
#apply it based on user_id
df_excluded.loc[:, 'snacks'] = df_excluded.groupby(['user_id'])['snacks'].transform('max')

In [72]:
#Set 'frozen' flag
df_excluded.loc[:, 'frozen'] = [1 if x == 'frozen' else 0 for x in df_excluded['department']]

In [73]:
#apply it based on user_id
df_excluded.loc[:, 'frozen'] = df_excluded.groupby(['user_id'])['frozen'].transform('max')

In [75]:
#Set 'alcohol' flag
df_excluded.loc[:, 'alcohol'] = [1 if x == 'alcohol' else 0 for x in df_excluded['department']]

In [76]:
#apply it based on user_id
df_excluded.loc[:, 'alcohol'] = df_excluded.groupby(['user_id'])['alcohol'].transform('max')

In [77]:
df_excluded['pet_items'].value_counts(dropna=False)

pet_items
0    27513213
1     3451351
Name: count, dtype: int64

In [78]:
df_excluded['baby_items'].value_counts(dropna=False)

baby_items
0    21154311
1     9810253
Name: count, dtype: int64

In [79]:
df_excluded['snacks'].value_counts(dropna=False)

snacks
1    29852553
0     1112011
Name: count, dtype: int64

In [80]:
df_excluded['frozen'].value_counts(dropna=False)

frozen
1    28887202
0     2077362
Name: count, dtype: int64

In [81]:
df_excluded['alcohol'].value_counts(dropna=False)

alcohol
0    28634074
1     2330490
Name: count, dtype: int64

In [82]:
df_excluded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30964564 entries, 0 to 32380882
Data columns (total 37 columns):
 #   Column                      Dtype   
---  ------                      -----   
 0   order_id                    int32   
 1   user_id                     int32   
 2   order_number                int8    
 3   orders_day_of_week          category
 4   order_hour_of_day           int8    
 5   days_since_prior_order      float16 
 6   product_id                  int32   
 7   add_to_cart_order           uint8   
 8   reordered                   int8    
 9   product_name                object  
 10  department_id               int8    
 11  prices                      float32 
 12  price_label                 object  
 13  busiest_days                object  
 14  max_order                   int8    
 15  loyalty_flag                object  
 16  mean_product_price          float32 
 17  spending_flag               object  
 18  median_days_between_orders  float16 
 19  ord

In [83]:
# I will create a few more additional customer profiles in 4.10 Part 2, need to save and downsample variables first.

# 06 Export Data

In [ ]:
#export df_all as pkl
#Export the updated df in pickle format as “ords_prods_customers_rev.pkl”. (incl sum_orders column)
df_all.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'instacart_ALL.pkl'))

In [ ]:
#export df_excluded as pkl
df_excluded.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'instacart_excluded.pkl'))

In [ ]:
#export df_low_activity customers sample as pkl
df_low_activity.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'instacart_low_activity_sample.pkl'))